## Automated PCSP Populator

In [1]:
import pandas as pd

In [2]:
#Import variables from 'feature_engineering' notebook
%store -r data
%store -r data_133

Automated PCSP Script with updated %'s

In [3]:
# script to get rid of curly brackets '{}':

text = '''De_Ply2Serve_2nd = pcase {                 
    9 : ServeT_in{ball= 1} -> Ply1_BackHandR
    0 : ServeT_err{ball=9} -> Fdoublefault{fscore++; if (fscore == 7) {won = player1}
        else { turn = (turn+1)%4 }
        } -> NextPt
    37 : ServeWide_in{ball =1} -> Ply1_ForeHandR
    2: ServeWide_err{ball=9} -> Fdoublefault{fscore++; if (fscore == 7) {won = player1} 
        else { turn = (turn+1)%4 }
        } -> NextPt
    48: ServeBody_in{ball=1} -> (Ply1_ForeHandR [] Ply1_BackHandR)
    4 : ServeBody_err{ball=9} -> Fdoublefault{fscore++; if (fscore == 7) {won = player1} 
        else { turn = (turn+1)%4 }
        } -> NextPt
        };
'''

for line in text.splitlines():
    line = line.replace('{','{{').replace('}','}}')
    print(line)

De_Ply2Serve_2nd = pcase {{                 
    9 : ServeT_in{{ball= 1}} -> Ply1_BackHandR
    0 : ServeT_err{{ball=9}} -> Fdoublefault{{fscore++; if (fscore == 7) {{won = player1}}
        else {{ turn = (turn+1)%4 }}
        }} -> NextPt
    37 : ServeWide_in{{ball =1}} -> Ply1_ForeHandR
    2: ServeWide_err{{ball=9}} -> Fdoublefault{{fscore++; if (fscore == 7) {{won = player1}} 
        else {{ turn = (turn+1)%4 }}
        }} -> NextPt
    48: ServeBody_in{{ball=1}} -> (Ply1_ForeHandR [] Ply1_BackHandR)
    4 : ServeBody_err{{ball=9}} -> Fdoublefault{{fscore++; if (fscore == 7) {{won = player1}} 
        else {{ turn = (turn+1)%4 }}
        }} -> NextPt
        }};


## Serve

In [4]:
# pd.options.display.max_columns=100

In [39]:
%store -r serve_cluster_agglom

In [42]:
serve_cluster_agglom
#need to change the column name to "Serve Cluster" (fix in sports_analytics_v5 notebook)

,Return Cluster,Player
0,1,Ivo Karlovic
1,1,John Isner
2,1,Reilly Opelka
3,1,Goran Ivanisevic
4,1,Milos Raonic
...,...,...
128,0,Mats Wilander
129,2,Sergi Bruguera
130,2,Yoshihito Nishioka
131,2,Jimmy Connors


In [7]:
# # serve_cluster_agglom
# serve_cluster_agglom[serve_cluster_agglom["Player"]=='Rafael Nadal']
# list(serve_cluster_agglom[serve_cluster_agglom["Return Cluster"]==2]["Player"])
# serve_cluster_agglom[serve_cluster_agglom["Player"]=='Roger Federer']
# list(serve_cluster_agglom[serve_cluster_agglom["Return Cluster"]==0]["Player"])

In [43]:
#create serve_133 df:
serve_133=data_133[data_133["Shot_Type_"].isin(["1st serve","2nd serve"])]
serve_133 = serve_133.drop(list(serve_133.index[serve_133["From_Court"]=='unknown']))
serve_133 = serve_133.drop(list(serve_133.index[serve_133['direction_outcome']\
    .isin(['unknown-fault','0-fault', 'unknown-no outcome', 'unknown-service winner + ace'])]))

In [44]:
def gen_serve_pcsp(player,court,serve_no, final_serve_dict):   
    opp_player = "Ply2" if player=="Ply1" else "Ply1" 
    land_court = 6 if (player=="Ply1" and court=="deuce")\
        else 1 if (player=="Ply2" and court=="deuce")\
        else 4 if (player=="Ply1" and court=="ad")\
        else 3 if (player=="Ply2" and court=="ad") else "ERROR"
    court_code = "De" if court=="deuce" else "Ad" if court=="ad" else "ERROR"
    opp_player_short = "n" if opp_player=="Ply2" else "f" if opp_player=="Ply1" else "ERROR"

    if serve_no==1:
        func_name = f"{court_code}_{player}Serve"
        parameter = f"1st serve-{court} court"
        print(f'''{func_name} = pcase {{
            {final_serve_dict[parameter]["server to T-over"]
                :.0f}: ServeT_in{{ball= {land_court}}} -> {opp_player}_{"BackHandR" if court=="deuce" else "ForeHandR" if court=="ad" else "ERROR"}
            {final_serve_dict[parameter]["server to T-not over"]
                :.0f}: ServeT_err{{ball=9}} -> {court_code}_{player}Serve_2nd
            {final_serve_dict[parameter]["serve to wide-over"]
                :.0f}: ServeWide_in{{ball ={land_court}}} -> {opp_player}_{"ForeHandR" if court=="deuce" else "BackHandR" if court=="ad" else "ERROR"}
            {final_serve_dict[parameter]["serve to wide-not over"]
                :.0f}: ServeWide_err{{ball=9}} -> {court_code}_{player}Serve_2nd
            {final_serve_dict[parameter]["serve to body-over"]
                :.0f}: ServeBody_in{{ball={land_court}}} -> ({opp_player}_BackHandR [] {opp_player}_ForeHandR)
            {final_serve_dict[parameter]["serve to body-not over"]
                :.0f}: ServeBody_err{{ball=9}} -> {court_code}_{player}Serve_2nd}}; 
        ''')
    elif serve_no==2:
        func_name = f"{court_code}_{player}Serve_2nd"
        parameter = f"2nd serve-{court} court"

        print(f'''{func_name} = pcase {{                 
            {final_serve_dict[parameter]["server to T-over"]:.0f} : ServeT_in{{ball= {land_court}}} -> {opp_player}_{"BackHandR" if court=="deuce" else "ForeHandR" if court=="ad" else "ERROR" }
            {final_serve_dict[parameter]["server to T-not over"]:.0f} : ServeT_err{{ball=9}} -> Fdoublefault{{{opp_player_short}score++; if ({opp_player_short}score == 7) {{won = {"player1" if opp_player=="Ply1" else "player2"}}}
                else {{ turn = (turn+1)%4 }}
                }} -> NextPt
            {final_serve_dict[parameter]["serve to wide-over"]:.0f} : ServeWide_in{{ball ={land_court}}} -> {opp_player}_{"ForeHandR" if court=="deuce" else "BackHandR" if court=="ad" else "ERROR" }
            {final_serve_dict[parameter]["serve to wide-not over"]:.0f}: ServeWide_err{{ball=9}} -> Fdoublefault{{{opp_player_short}score++; if ({opp_player_short}score == 7) {{won = {"player1" if opp_player=="Ply1" else "player2" if opp_player=="Ply2" else "ERROR"}}} 
                else {{ turn = (turn+1)%4 }}
                }} -> NextPt
            {final_serve_dict[parameter]["serve to body-over"]:.0f}: ServeBody_in{{ball={land_court}}} -> ({opp_player}_ForeHandR [] {opp_player}_BackHandR)
            {final_serve_dict[parameter]["serve to body-not over"]:.0f} : ServeBody_err{{ball=9}} -> Fdoublefault{{{opp_player_short}score++; if ({opp_player_short}score == 7) {{won = {"player1" if opp_player=="Ply1" else "player2" if opp_player=="Ply2" else "ERROR"}}} 
                else {{ turn = (turn+1)%4 }}
                }} -> NextPt
                }};
        ''')
    else:
        print("error")

In [45]:
# serve_df[serve_df["From_Court"]=="ad court"]

# def df_v6(df:pd.DataFrame)-> pd.DataFrame:

# df = serve_df

def create_df_gen_pcsp(df,p1,p2,cluster=True):
    df=df[df["Shot_Type_"].isin(["1st serve","2nd serve"])]
    df = df.drop(list(df.index[serve_133["From_Court"]=='unknown']))
    df = df.drop(list(df.index[serve_133['direction_outcome'].isin(['unknown-fault','0-fault', 'unknown-no outcome', 'unknown-service winner + ace'])]))
    
    if not cluster:
        player_v_player_df = df[((df["Player 1 name"] == p1 )&(df["Player 2 name"] == p2))]

    # player="Rafael Nadal"
    # opp_player="Roger Federer"
    
    elif cluster:
        cluster_no = serve_cluster_agglom[serve_cluster_agglom["Player"]==p1]["Return Cluster"].values[0]
        opp_cluster_no = serve_cluster_agglom[serve_cluster_agglom["Player"]==p2]["Return Cluster"].values[0]

        player_cluster_df = df[df["Player 1 name"].isin(list(serve_cluster_agglom[serve_cluster_agglom["Return Cluster"]==cluster_no]["Player"]))]
        player_v_opp_player_df = player_cluster_df[player_cluster_df["Player 2 name"].isin(list(serve_cluster_agglom[serve_cluster_agglom["Return Cluster"]==opp_cluster_no]["Player"]))]

    def make_serve_dict(df):
        # df = player_v_opp_player_df
        shot_type = list(df['Shot_Type_'].unique())
        from_court = list(df['From_Court'].unique())
    # direction_outcome = list(data_133['direction_outcome'].unique())
        temp_dict = {}
        for st in shot_type:
            for fc in from_court:
                temp_df = df[(df['Shot_Type_']==st) & (df['From_Court']== fc)]\
                .groupby(['Shot_Type_','From_Court','serveDirection_outcome_basic']).count()\
                .transform(lambda x: x/x.sum())["Player 1 name"]
                temp_dict["{}-{}".format(st,fc)] = temp_df
        overall_df = pd.concat(temp_dict.values())
        overall_df = pd.DataFrame(overall_df).reset_index()

        serves = list(overall_df["Shot_Type_"].unique())
        courts = list(overall_df["From_Court"].unique())
        serve_dict={}
        for serve in serves: 
            for court in courts:
                serve_dict["{}-{}".format(serve,court)]=(overall_df[(overall_df["Shot_Type_"]==serve) & (overall_df["From_Court"]==court)])
        serve_types = ["1st serve-deuce court","2nd serve-deuce court","1st serve-ad court","2nd serve-ad court"]
        final_serve_dict={}
        for serve_type in serve_types:
            temp_dict={}
            for i in list(serve_dict[serve_type]["serveDirection_outcome_basic"].unique()):
                # print(i)
                current_df = serve_dict[serve_type]
                # print(current_df[current_df["serveDirection_outcome_basic"]==i]["Player 1 name"])
                temp_dict[i] = current_df[current_df["serveDirection_outcome_basic"]==i]["Player 1 name"].values[0]*100
            final_serve_dict[serve_type]=temp_dict
        return final_serve_dict

    if cluster:
        final_serve_dict = make_serve_dict(player_v_opp_player_df)
    elif not cluster:
        final_serve_dict=make_serve_dict(player_v_player_df)

    print(p1 + ':\n')
    # print(final_serve_dict)
    
    gen_serve_pcsp(player="Ply1", court="deuce",serve_no=1, final_serve_dict=final_serve_dict)
    gen_serve_pcsp(player="Ply1", court="deuce",serve_no=2, final_serve_dict=final_serve_dict)
    gen_serve_pcsp(player="Ply1", court="ad", serve_no=1, final_serve_dict=final_serve_dict)
    gen_serve_pcsp(player="Ply1", court="ad", serve_no=2, final_serve_dict=final_serve_dict)
    
    if cluster:
        player_cluster_df = df[df["Player 1 name"].isin(list(serve_cluster_agglom[serve_cluster_agglom["Return Cluster"]==opp_cluster_no]["Player"]))]
        player_v_opp_player_df = player_cluster_df[player_cluster_df["Player 2 name"].isin(list(serve_cluster_agglom[serve_cluster_agglom["Return Cluster"]==cluster_no]["Player"]))]
        final_serve_dict = make_serve_dict(player_v_opp_player_df)
    elif not cluster:
        player_v_player_df = df[((df["Player 1 name"] == p2 )&(df["Player 2 name"] == p1))]
        final_serve_dict=make_serve_dict(player_v_player_df)
        final_serve_dict = make_serve_dict(player_v_player_df)

    print(p2+':\n')
    gen_serve_pcsp(player="Ply2", court="deuce",serve_no=1, final_serve_dict=final_serve_dict)
    gen_serve_pcsp(player="Ply2", court="deuce",serve_no=2, final_serve_dict=final_serve_dict)
    gen_serve_pcsp(player="Ply2", court="ad", serve_no=1, final_serve_dict=final_serve_dict)
    gen_serve_pcsp(player="Ply2", court="ad", serve_no=2, final_serve_dict=final_serve_dict)

In [73]:
create_df_gen_pcsp(serve_133,"Roger Federer","Rafael Nadal",cluster=True)

# create_df_gen_pcsp(serve_133,"Roger Federer","Rafael Nadal",cluster=False)


Roger Federer:

De_Ply1Serve = pcase {
            26: ServeT_in{ball= 6} -> Ply2_BackHandR
            20: ServeT_err{ball=9} -> De_Ply1Serve_2nd
            32: ServeWide_in{ball =6} -> Ply2_ForeHandR
            15: ServeWide_err{ball=9} -> De_Ply1Serve_2nd
            5: ServeBody_in{ball=6} -> (Ply2_BackHandR [] Ply2_ForeHandR)
            2: ServeBody_err{ball=9} -> De_Ply1Serve_2nd}; 
        
De_Ply1Serve_2nd = pcase {                 
            23 : ServeT_in{ball= 6} -> Ply2_BackHandR
            2 : ServeT_err{ball=9} -> Fdoublefault{nscore++; if (nscore == 7) {won = player2}
                else { turn = (turn+1)%4 }
                } -> NextPt
            41 : ServeWide_in{ball =6} -> Ply2_ForeHandR
            4: ServeWide_err{ball=9} -> Fdoublefault{nscore++; if (nscore == 7) {won = player2} 
                else { turn = (turn+1)%4 }
                } -> NextPt
            28: ServeBody_in{ball=6} -> (Ply2_ForeHandR [] Ply2_BackHandR)
            2 : ServeBody_err{bal

Find a cluster that is Not Nadal nor Federer, and use that cluster inplace of Nadal's (Ply2's) cluster, to act as a comparison point / control variable?

In [80]:
serve_cluster_agglom[serve_cluster_agglom["Player"].isin(['Rafael Nadal','Roger Federer'])]

,Return Cluster,Player
21,3,Roger Federer
104,2,Rafael Nadal


In [100]:
serve_cluster_agglom
# serve_cluster_agglom[serve_cluster_agglom["Player"]=="Rafael Nadal"]

,Return Cluster,Player
0,1,Ivo Karlovic
1,1,John Isner
2,1,Reilly Opelka
3,1,Goran Ivanisevic
4,1,Milos Raonic
...,...,...
128,0,Mats Wilander
129,2,Sergi Bruguera
130,2,Yoshihito Nishioka
131,2,Jimmy Connors


We use Mats Wilander for Cluster 0, so we set Federer's cluster vs Wiland, or John Isner for cluster 1.
Because there can be too many combinations, given 3 game components, we will choose a 'Non-Ply2 cluster' by random for each Game Component. Given enough time, we could test more combinations.

In [106]:
# create_df_gen_pcsp(serve_133,"John Isner","Rafael Nadal",cluster=True)
#Cluster 1 vs 0
create_df_gen_pcsp(serve_133,"John Isner","Mats Wilander",cluster=True) 

John Isner:

De_Ply1Serve = pcase {
            30: ServeT_in{ball= 6} -> Ply2_BackHandR
            17: ServeT_err{ball=9} -> De_Ply1Serve_2nd
            29: ServeWide_in{ball =6} -> Ply2_ForeHandR
            14: ServeWide_err{ball=9} -> De_Ply1Serve_2nd
            7: ServeBody_in{ball=6} -> (Ply2_BackHandR [] Ply2_ForeHandR)
            2: ServeBody_err{ball=9} -> De_Ply1Serve_2nd}; 
        
De_Ply1Serve_2nd = pcase {                 
            32 : ServeT_in{ball= 6} -> Ply2_BackHandR
            5 : ServeT_err{ball=9} -> Fdoublefault{nscore++; if (nscore == 7) {won = player2}
                else { turn = (turn+1)%4 }
                } -> NextPt
            19 : ServeWide_in{ball =6} -> Ply2_ForeHandR
            2: ServeWide_err{ball=9} -> Fdoublefault{nscore++; if (nscore == 7) {won = player2} 
                else { turn = (turn+1)%4 }
                } -> NextPt
            39: ServeBody_in{ball=6} -> (Ply2_ForeHandR [] Ply2_BackHandR)
            3 : ServeBody_err{ball=9

In [ ]:
create_df_gen_pcsp(serve_133,"Roger Federer","Mats Wilander",cluster=True)

### data checks

In [12]:
def make_serve_dict(df):
    # df = player_v_opp_player_df
    shot_type = list(df['Shot_Type_'].unique())
    from_court = list(df['From_Court'].unique())
# direction_outcome = list(data_133['direction_outcome'].unique())
    temp_dict = {}
    for st in shot_type:
        for fc in from_court:
            temp_df = df[(df['Shot_Type_']==st) & (df['From_Court']== fc)]\
            .groupby(['Shot_Type_','From_Court','serveDirection_outcome_basic']).count()\
            .transform(lambda x: x/x.sum())["Player 1 name"]
            temp_dict["{}-{}".format(st,fc)] = temp_df
    overall_df = pd.concat(temp_dict.values())
    overall_df = pd.DataFrame(overall_df).reset_index()

    serves = list(overall_df["Shot_Type_"].unique())
    courts = list(overall_df["From_Court"].unique())
    serve_dict={}
    for serve in serves: 
        for court in courts:
            serve_dict["{}-{}".format(serve,court)]=(overall_df[(overall_df["Shot_Type_"]==serve) & (overall_df["From_Court"]==court)])
    # serve_dict

    serve_types = ["1st serve-deuce court","2nd serve-deuce court","1st serve-ad court","2nd serve-ad court"]

    final_serve_dict={}
    for serve_type in serve_types:
        temp_dict={}
        for i in list(serve_dict[serve_type]["serveDirection_outcome_basic"].unique()):
            # print(i)
            current_df = serve_dict[serve_type]
            # print(current_df[current_df["serveDirection_outcome_basic"]==i]["Player 1 name"])
            temp_dict[i] = current_df[current_df["serveDirection_outcome_basic"]==i]["Player 1 name"].values[0]*100
        final_serve_dict[serve_type]=temp_dict
    return final_serve_dict

In [13]:
# data checks

p1="Roger Federer"
p2="Rafael Nadal"
cluster_no = serve_cluster_agglom[serve_cluster_agglom["Player"]==p1]["Return Cluster"].values[0]
opp_cluster_no = serve_cluster_agglom[serve_cluster_agglom["Player"]==p2]["Return Cluster"].values[0]

player_cluster_df = serve_133[serve_133["Player 1 name"].isin(list(serve_cluster_agglom[serve_cluster_agglom["Return Cluster"]==cluster_no]["Player"]))]
player_v_opp_player_df = player_cluster_df[player_cluster_df["Player 2 name"].isin(list(serve_cluster_agglom[serve_cluster_agglom["Return Cluster"]==opp_cluster_no]["Player"]))]

make_serve_dict(player_v_opp_player_df)



{'1st serve-deuce court': {'serve to body-not over': 1.9592208679804077,
  'serve to body-over': 5.251167558947489,
  'serve to wide-not over': 15.286479097847137,
  'serve to wide-over': 31.9398564756806,
  'server to T-not over': 19.888369973801115,
  'server to T-over': 25.674906025743248},
 '2nd serve-deuce court': {'serve to body-not over': 2.3284313725490198,
  'serve to body-over': 27.66544117647059,
  'serve to wide-not over': 3.553921568627451,
  'serve to wide-over': 41.42156862745098,
  'server to T-not over': 2.2058823529411766,
  'server to T-over': 22.824754901960784},
 '1st serve-ad court': {'serve to body-not over': 1.8263697773329997,
  'serve to body-over': 5.741806354766075,
  'serve to wide-not over': 23.380035026269702,
  'serve to wide-over': 27.433074806104578,
  'server to T-not over': 15.286464848636477,
  'server to T-over': 26.332249186890166},
 '2nd serve-ad court': {'serve to body-not over': 1.726263871763255,
  'serve to body-over': 23.42786683107275,
  's

## Return

In [53]:
%store -r return_cluster_agglom
%store -r return_cluster_kmeans

In [54]:
return_cluster_kmeans
# return_cluster_agglom[return_cluster_agglom["Player"]=='Rafael Nadal']
# list(return_cluster_agglom[return_cluster_agglom["Return Cluster"]==0]["Player"])

,Return Cluster,Player
0,1,Guillermo Coria
1,3,Mats Wilander
2,1,Alejandro Davidovich Fokina
3,1,Marton Fucsovics
4,3,Sergi Bruguera
...,...,...
128,0,John Isner
129,0,Reilly Opelka
130,2,Goran Ivanisevic
131,2,Ivo Karlovic


In [16]:
# pd.options.display.max_columns=100
# data_133.head()
return_133 = data_133[data_133["Shot_Type_"]=="return"]
len(return_133)

286457

In [17]:
# return_133.rally_outcome.value_counts()
# return_133[return_133["rally_outcome"]=="BH_None"]
# return_133.groupby(['rally_outcome','outcome_basic'])["Player 1 name"].count()
# return_133.head()

In [18]:
# return_133.head()
# # return_133[return_133["From_Court"]=='middle court']["Prev shot From which court"].value_counts()
# # "Prev shot From which court"


In [19]:
# return_c0 = return_133[return_133["Player 1 name"].isin(list(return_cluster_agglom[return_cluster_agglom["Return Cluster"]==0]["Player"]))]
# return_c0.head()


In [20]:
# return_nadal = return_133[return_133["Player 1 name"].str.contains('Nada')]
# return_nadal

In [21]:
# player_v_opp_player_df.head()

#Nadal's cluster vs Fed's cluster

In [69]:
# Filtering the DF to be Player 1's cluster vs Player 2's cluster:

#simply replacing the agglom results variable with the kmeans result, based on best eval results in prev stage.
return_cluster_agglom = return_cluster_kmeans

def filter_df_return(player,opp_player,return_133):
    # player="Roger Federer"
    # opp_player="Rafael Nadal"
    cluster_no = return_cluster_agglom[return_cluster_agglom["Player"]==player]["Return Cluster"].values[0]
    opp_cluster_no = return_cluster_agglom[return_cluster_agglom["Player"]==opp_player]["Return Cluster"].values[0]

    player_cluster_df = return_133[return_133["Player 1 name"].isin(list(return_cluster_agglom[return_cluster_agglom["Return Cluster"]==cluster_no]["Player"]))]
    player_v_opp_player_df = player_cluster_df[player_cluster_df["Player 2 name"].isin(list(return_cluster_agglom[return_cluster_agglom["Return Cluster"]==opp_cluster_no]["Player"]))]

    #Define a filtered DF for each cluster in place of the return_133 df here:

    cluster_df = player_v_opp_player_df

    # cluster_df["rally_outcome"].value_counts()
    # for idx,name in enumerate(df['Surface'].value_counts().index.tolist()):
    return_dc={}
    for i,name in enumerate(cluster_df[cluster_df["Prev shot From which court"]==1]["rally_outcome"].value_counts().index.tolist()):
        # print(name)    
        # print(return_133[return_133["Prev shot From which court"]==1]["rally_outcome"].value_counts()[i])
        # print(cluster_df[cluster_df["Prev shot From which court"]==1]["rally_outcome"].value_counts(normalize=True)[i])
        return_dc[name]=cluster_df[cluster_df["Prev shot From which court"]==1]["rally_outcome"].value_counts(normalize=True)[i]*100

    return_ac={}
    for i,name in enumerate(cluster_df[cluster_df["Prev shot From which court"]==3]["rally_outcome"].value_counts().index.tolist()):
        # print(name)    
        # print(return_133[return_133["Prev shot From which court"]==1]["rally_outcome"].value_counts()[i])
        # print(cluster_df[cluster_df["Prev shot From which court"]==3]["rally_outcome"].value_counts(normalize=True)[i])
        return_ac[name]=cluster_df[cluster_df["Prev shot From which court"]==3]["rally_outcome"].value_counts(normalize=True)[i]*100
    # print(return_dc)
    # print(return_ac)
    return(return_dc,return_ac)

In [70]:
def gen_return_pcsp(player, return_dc, return_ac):

# player="Ply2"
    opp_player = "Ply2" if player=="Ply1" else "Ply1" if player=="Ply2" else "ERROR"
    opp_player_long = "player2" if opp_player=="Ply2" else "player1" if opp_player=="Ply1" else "ERROR"
    opp_player_short = "n" if opp_player=="Ply2" else "f" if opp_player=="Ply1" else "ERROR"

    process_name = f'{player}_BackHandR'

    if player=="Ply1":
        bh_insidein = 4
        bh_insideout=6
        bh_downmid=5
        bh_crosscourt=4
        bh_downline=6
        bh_downmid=5
    elif player=="Ply2":
        bh_insidein = 3
        bh_insideout=1
        bh_downmid=2
        bh_crosscourt=3
        bh_downline=1
        bh_downmid=2


    print(f'''{process_name} = ifa (turn == {opp_player_short}_de_ct) {{   
        pcase {{
        {return_dc["BH_inside_out"]:.0f}: BH_InsideIn {{ball = {bh_insidein}}} -> {opp_player}_ad_stroke
        {return_dc["BH_inside_in"]:.0f}: BH_InsideOut {{ ball = {bh_insideout}}} -> {opp_player}_de_stroke
        {return_dc['BH_down_mid']:.0f}: BH_DownMid {{ ball = {bh_downmid}}} -> {opp_player}_mid_stroke
        {return_dc["BH_Error"]:.0f}: BH_Error {{ ball = 9}} -> {{{opp_player_short}score++; if ({opp_player_short}score == 7) {{won = {opp_player_long}}}
            else {{ turn = (turn+1)%4 }}
            }} -> NextPt
        }}}}
            else {{
        pcase {{
            {return_ac["BH_cross_court"]:.0f}: BH_CrossCourt {{ ball = {bh_crosscourt}}} -> {opp_player}_ad_stroke
            {return_ac["BH_down_line"]:.0f}: BH_DownLine {{ ball = {bh_downline}}} -> {opp_player}_de_stroke
            {return_ac["BH_down_mid"]:.0f}: BH_DownMid {{ ball = {bh_downmid} }} -> {opp_player}_mid_stroke
            {return_ac["BH_Error"]:.0f}: BH_Error{{ball=9}} -> {{{opp_player_short}score++; if ({opp_player_short}score == 7) {{won = {opp_player_long}}} 
                else {{ turn = (turn+1)%4 }}
                }} -> NextPt
            }}}};
        ''')

    process_name = f'{player}_ForeHandR'
    if player=="Ply1":
        fh_crosscourt = 6
        fh_downline=4
        fh_downmid=5
        fh_insideout=4
        fh_insidein=6
        fh_downmid=5
    elif player=="Ply2":
        fh_crosscourt = 1
        fh_downline=3
        fh_downmid=2
        fh_insideout=3
        fh_insidein=1
        fh_downmid=2

    print(f'''{process_name} = ifa (turn == {opp_player_short}_de_ct) {{    
        pcase {{
        {return_dc["FH_cross_court"]:.0f}: FH_CrossCourt {{ ball = {fh_crosscourt};}} -> {opp_player}_de_stroke
        {return_dc["FH_down_line"]:.0f}: FH_DownLine {{ ball = {fh_downline}}} -> {opp_player}_ad_stroke
        {return_dc["FH_down_mid"]:.0f}: FH_DownMid {{ ball = {fh_downmid}}}-> {opp_player}_mid_stroke
        {return_dc["FH_Error"]:.0f}: FH_Error {{ ball = 9}} -> {{{opp_player_short}score++; if ({opp_player_short}score == 7) {{won = {opp_player_long}}}
            else {{ turn = (turn+1)%4 }}
            }} -> NextPt
        }}}}
            else {{
        pcase {{
            {return_ac["FH_inside_out"]:.0f}: FH_InsideOut {{ ball = {fh_insideout}}} -> {opp_player}_ad_stroke
            {return_ac["FH_inside_in"]:.0f}: FH_InsideIn {{ ball = {fh_insidein}}} -> {opp_player}_de_stroke
            {return_ac["FH_down_mid"]:.0f}: FH_DownMid {{ ball = {fh_downmid}}} -> {opp_player}_mid_stroke
            {return_ac["FH_Error"]:.0f}: FH_Error {{ ball = 9}} -> {{{opp_player_short}score++; if ({opp_player_short}score == 7) {{won = {opp_player_long}}} 
                else {{ turn = (turn+1)%4 }}
                }} -> NextPt
             }}}};
    ''')

In [71]:
def wrapper_return_pcsp(p1,p2,df):
    print(p1 + ":\n")
    return_dc,return_ac= filter_df_return(p1,p2,df)
    gen_return_pcsp('Ply1',return_dc,return_ac)
    print(p2 + ":\n")    
    return_dc,return_ac= filter_df_return(p2,p1,df)
    gen_return_pcsp('Ply2',return_dc,return_ac)

In [74]:
wrapper_return_pcsp('Roger Federer','Rafael Nadal',return_133)

Roger Federer:

Ply1_BackHandR = ifa (turn == n_de_ct) {   
        pcase {
        9: BH_InsideIn {ball = 4} -> Ply2_ad_stroke
        3: BH_InsideOut { ball = 6} -> Ply2_de_stroke
        29: BH_DownMid { ball = 5} -> Ply2_mid_stroke
        10: BH_Error { ball = 9} -> {nscore++; if (nscore == 7) {won = player2}
            else { turn = (turn+1)%4 }
            } -> NextPt
        }}
            else {
        pcase {
            22: BH_CrossCourt { ball = 4} -> Ply2_ad_stroke
            3: BH_DownLine { ball = 6} -> Ply2_de_stroke
            23: BH_DownMid { ball = 5 } -> Ply2_mid_stroke
            11: BH_Error{ball=9} -> {nscore++; if (nscore == 7) {won = player2} 
                else { turn = (turn+1)%4 }
                } -> NextPt
            }};
        
Ply1_ForeHandR = ifa (turn == n_de_ct) {    
        pcase {
        9: FH_CrossCourt { ball = 6;} -> Ply2_de_stroke
        4: FH_DownLine { ball = 4} -> Ply2_ad_stroke
        20: FH_DownMid { ball = 5}-> Ply2_mid_stroke

Find a cluster that is Not Nadal nor Federer, and use that cluster inplace of Nadal's (Ply2's) cluster, to act as a comparison point / control variable?

In [84]:
return_cluster_agglom[return_cluster_agglom["Player"].isin(['Rafael Nadal','Roger Federer'])]

,Return Cluster,Player
7,1,Rafael Nadal
49,1,Roger Federer


In [89]:
return_cluster_agglom

,Return Cluster,Player
0,1,Guillermo Coria
1,3,Mats Wilander
2,1,Alejandro Davidovich Fokina
3,1,Marton Fucsovics
4,3,Sergi Bruguera
...,...,...
128,0,John Isner
129,0,Reilly Opelka
130,2,Goran Ivanisevic
131,2,Ivo Karlovic


We use John Isner for cluster 0.
Because there can be too many combinations, given 3 game components, we will choose a 'Non-Ply2 cluster' by random for each Game Component. Given enough time, we could test more combinations.

Karlovic is cluster 2.

In [ ]:
# wrapper_return_pcsp('John Isner','Rafael Nadal',return_133)
wrapper_return_pcsp('John Isner','Ivo Karlovic',return_133)

John Isner:

Ply1_BackHandR = ifa (turn == n_de_ct) {   
        pcase {
        4: BH_InsideIn {ball = 4} -> Ply2_ad_stroke
        3: BH_InsideOut { ball = 6} -> Ply2_de_stroke
        32: BH_DownMid { ball = 5} -> Ply2_mid_stroke
        16: BH_Error { ball = 9} -> {nscore++; if (nscore == 7) {won = player2}
            else { turn = (turn+1)%4 }
            } -> NextPt
        }}
            else {
        pcase {
            22: BH_CrossCourt { ball = 4} -> Ply2_ad_stroke
            3: BH_DownLine { ball = 6} -> Ply2_de_stroke
            21: BH_DownMid { ball = 5 } -> Ply2_mid_stroke
            12: BH_Error{ball=9} -> {nscore++; if (nscore == 7) {won = player2} 
                else { turn = (turn+1)%4 }
                } -> NextPt
            }};
        
Ply1_ForeHandR = ifa (turn == n_de_ct) {    
        pcase {
        5: FH_CrossCourt { ball = 6;} -> Ply2_de_stroke
        6: FH_DownLine { ball = 4} -> Ply2_ad_stroke
        14: FH_DownMid { ball = 5}-> Ply2_mid_stroke
  

### data checks

In [26]:
#data checks

player = "Roger Federer"
opp_player="Rafael Nadal"

cluster_no = return_cluster_agglom[return_cluster_agglom["Player"]==player]["Return Cluster"].values[0]
opp_cluster_no = return_cluster_agglom[return_cluster_agglom["Player"]==opp_player]["Return Cluster"].values[0]

player_cluster_df = return_133[return_133["Player 1 name"].isin(list(return_cluster_agglom[return_cluster_agglom["Return Cluster"]==cluster_no]["Player"]))]
player_v_opp_player_df = player_cluster_df[player_cluster_df["Player 2 name"].isin(list(return_cluster_agglom[return_cluster_agglom["Return Cluster"]==opp_cluster_no]["Player"]))]


In [27]:
for i,name in enumerate(player_v_opp_player_df[player_v_opp_player_df["Prev shot From which court"]==3]["rally_outcome"].value_counts().index.tolist()):
    print(name)
    print(player_v_opp_player_df[player_v_opp_player_df["Prev shot From which court"]==3]["rally_outcome"].value_counts(normalize=True)[i]*100)

BH_down_mid
24.2469174340565
BH_cross_court
23.232402060246603
FH_down_mid
16.216638052130484
BH_Error
12.166380521304824
FH_Error
8.436085531449978
FH_inside_out
7.850788200405806
BH_down_line
3.4181364132979555
FH_cross_court
1.685656313407211
FH_inside_in
1.2252224129857967
BH_None
0.9286717652567505
FH_None
0.3980021851100359
BH_inside_out
0.13266739503667863
-
0.062431715311378186


## Rally

In [61]:
%store -r rally_cluster_agglom
# %store -r rally_cluster_kmeans

In [62]:
rally_133=data_133[data_133["Shot_Type_"].isin(["rally"])]
rally_133.head()

Short,Player 1 name,Player 2 name,Player 1 handness,Player 2 handness,Player 1 points,Player 2 points,Player 1 games,Player 2 games,Player 1 sets,Player 2 sets,...,To which court_,shot_group,Year,match,direction_2,rally_outcome,return_direction_outcome,outcome_basic,return_direction_outcome_2,serveDirection_outcome_basic
2,Novak Djokovic,Alexander Zverev,RH,RH,0,0,0,0,0,0,...,deuce court,forehand,2018,0 2018-11-18\n1 2018-11-18\n2 ...,cross_court,FH_cross_court,deuce court-no outcome,over,deuce court-over,7-over
3,Alexander Zverev,Novak Djokovic,RH,RH,0,0,0,0,0,0,...,middle court,forehand,2018,0 2018-11-18\n1 2018-11-18\n2 ...,down_mid,FH_down_mid,middle court-no outcome,over,middle court-over,middle court-over
4,Novak Djokovic,Alexander Zverev,RH,RH,0,0,0,0,0,0,...,middle court,forehand,2018,0 2018-11-18\n1 2018-11-18\n2 ...,down_mid,FH_Error,middle court-unforced error,not over,middle court-not over,middle court-not over
7,Novak Djokovic,Alexander Zverev,RH,RH,0,1,0,0,0,0,...,middle court,backhand,2018,0 2018-11-18\n1 2018-11-18\n2 ...,down_mid,BH_down_mid,middle court-no outcome,over,middle court-over,middle court-over
8,Alexander Zverev,Novak Djokovic,RH,RH,1,0,0,0,0,0,...,deuce court,forehand,2018,0 2018-11-18\n1 2018-11-18\n2 ...,cross_court,FH_cross_court,deuce court-no outcome,over,deuce court-over,7-over


In [63]:
def filter_df_rally(p1,p2,rally_133):
    # Filtering the DF to be Player 1's cluster vs Player 2's cluster:
    # player="Rafael Nadal"
    # player=p1
    # opp_player="Roger Federer"
    cluster_no = rally_cluster_agglom[rally_cluster_agglom["Player"]==p1]["Return Cluster"].values[0]
    opp_cluster_no = rally_cluster_agglom[rally_cluster_agglom["Player"]==p2]["Return Cluster"].values[0]

    player_cluster_df = rally_133[rally_133["Player 1 name"].isin(list(rally_cluster_agglom[rally_cluster_agglom["Return Cluster"]==cluster_no]["Player"]))]
    player_v_opp_player_df = player_cluster_df[player_cluster_df["Player 2 name"].isin(list(rally_cluster_agglom[rally_cluster_agglom["Return Cluster"]==opp_cluster_no]["Player"]))]
    player_v_opp_player_df.head()

    #Nadal's cluster vs Fed's cluster

    #Creating the Rally df

    # df = rally_133
    df = player_v_opp_player_df

    df = df[df["Shot_Type_"].isin(["rally"])]
    df = df.drop(list(df.index[df["From_Court"]=='unknown']))
    df = df.drop(list(df.index[df['rally_outcome'].str.contains("None|-")]))
    # print(list(df["rally_outcome"].unique()))
    from_court = list(df['From_Court'].unique())
    # print(from_court)
    temp_dict = {}
    for fc in from_court:
        temp_df = df[df['From_Court']== fc].groupby(['From_Court','rally_outcome']).size().transform(lambda x: x/x.sum())
        temp_dict["{}".format(fc)] = temp_df
    #         print(test_df[(test_df['Shot_Type_']==st) & (test_df['From_Court']== fc)]\
    #         .groupby(['Shot_Type_','From_Court','direction_outcome']).count()["Player 1 name"])            

    overall_df = pd.concat(temp_dict.values())
    overall_df = pd.DataFrame(overall_df).reset_index()
    overall_df.columns=["From_Court","rally_outcome","percentage"]
    # print(overall_df)
    return overall_df

In [64]:
def gen_rally_pcsp(player,court,overall_df):
    opp_player = "Ply2" if player=="Ply1" else "Ply1" if player=="Ply2" else "ERROR"
    # court_code = "De" if court=="deuce" else "Ad" if court=="ad" else "ERROR"
    opp_player_long = "player2" if opp_player=="Ply2" else "player1" if opp_player=="Ply1" else "ERROR"
    opp_player_short = "n" if opp_player=="Ply2" else "f" if opp_player=="Ply1" else "ERROR"
    
    if court not in ["ad","de","mid"]:
        print("ERROR")
        return
    if player not in ["Ply1","Ply2"]:
        print("ERROR")
        return
    
    func_name = f"{player}_{court}_stroke"
    
    parameter = f"1st serve-{court} court"
    
    def get_perc(shot_type):
        return float(temp_df[temp_df["rally_outcome"]==shot_type]["percentage"])*100    
    
    if func_name == f"{player}_de_stroke":
        temp_df = overall_df[overall_df["From_Court"]=="deuce court"]        
        if player=="Ply1":
            fh_crosscourt = 6
            fh_downline=4
            fh_downmid=5
            bh_insidein=4
            bh_insideout=6
            bh_downmid=5
        elif player=="Ply2":
            fh_crosscourt = 1
            fh_downline=3
            fh_downmid=2
            bh_insidein=3
            bh_insideout=1
            bh_downmid=2

        print(f'''{func_name} = pcase{{
            {get_perc("FH_cross_court"):.0f}: FH_Crosscourt {{ ball = {fh_crosscourt}}} -> {opp_player}_de_stroke
            {get_perc("FH_down_line"):.0f}: FH_Downline {{ ball = {fh_downline}}} -> {opp_player}_ad_stroke
            {get_perc("FH_down_mid"):.0f}: FH_DownMid {{ ball = {fh_downmid}}} -> {opp_player}_mid_stroke
            {get_perc("BH_inside_in"):.0f}: BH_InsideIn {{ball = {bh_insidein}}} -> {opp_player}_ad_stroke
            {get_perc("BH_inside_out"):.0f}: BH_InsideOut {{ ball = {bh_insideout}}} -> {opp_player}_de_stroke
            {get_perc("BH_down_mid"):.0f}: BH_DownMid {{ ball = {bh_downmid}}} -> Ply2_mid_stroke
            {get_perc("FH_Error"):.0f}: FH_Error {{ ball = 9}} -> {{{opp_player_short}score++; if ({opp_player_short}score == 7) {{won = {opp_player_long}}}
                else {{ turn = (turn+1)%4 }}
                }} -> NextPt
            {get_perc("BH_Error"):.0f}: BH_Error {{ ball = 9}} -> {{{opp_player_short}score++; if ({opp_player_short}score == 7) {{won = {opp_player_long}}}
                else {{ turn = (turn+1)%4 }}
                }} -> NextPt
            }};
            ''')
        
    if func_name == f"{player}_mid_stroke":
        temp_df = overall_df[overall_df["From_Court"]=="middle court"]
        if player=="Ply1":
            fh_insideout=4
            fh_crosscourt=6
            fh_downmid=5
            bh_insideout=6
            bh_downmid=5
            bh_crosscourt=4
        elif player=="Ply2":
            fh_insideout=3
            fh_crosscourt=1
            fh_downmid=2
            bh_insideout=1
            bh_downmid=2
            bh_crosscourt=3
            
        print(f'''{func_name} = pcase{{
            {get_perc("FH_inside_out"):.0f}: FH_InsideOut{{ball = {fh_insideout}}} -> {opp_player}_ad_stroke
            {get_perc("FH_cross_court"):.0f}: FH_Crosscourt {{ ball = {fh_crosscourt}}} -> {opp_player}_de_stroke
            {get_perc("FH_down_mid"):.0f}: FH_DownMid {{ ball = {fh_downmid} }} -> {opp_player}_mid_stroke
            {get_perc("BH_inside_out"):.0f}: BH_InsideOut {{ ball = {bh_insideout}}} -> {opp_player}_de_stroke
            {get_perc("BH_down_mid"):.0f}: BH_DownMid {{ ball = {bh_downmid}}} -> {opp_player}_mid_stroke
            {get_perc("BH_cross_court"):.0f}: BH_Crosscourt {{ ball = {bh_crosscourt}}} -> {opp_player}_ad_stroke
            {get_perc("FH_Error"):.0f}: FH_Error {{ ball = 9}} -> {{{opp_player_short}score++; if ({opp_player_short}score == 7) {{won = {opp_player_long}}}
                else {{ turn = (turn+1)%4 }}
                }} -> NextPt
            {get_perc("BH_Error"):.0f}: BH_Error {{ ball = 9}} -> {{{opp_player_short}score++; if ({opp_player_short}score == 7) {{won = {opp_player_long}}}
                else {{ turn = (turn+1)%4 }}
                }} -> NextPt   
            }};
            ''')
        
    if func_name == f"{player}_ad_stroke":
        temp_df = overall_df[overall_df["From_Court"]=="ad court"]        
        if player=="Ply1":
            bh_crosscourt=4
            bh_downline=6
            bh_downmid=5
            fh_insideout=4
            fh_insidein=6
            fh_downmid=5
        elif player=="Ply2":
            bh_crosscourt=3
            bh_downline=1
            bh_downmid=2
            fh_insideout=3
            fh_insidein=1
            fh_downmid=2
            
        print(f'''{func_name} = pcase{{
                {get_perc("BH_cross_court"):.0f}: BH_Crosscourt {{ ball = {bh_crosscourt}}} -> {opp_player}_ad_stroke
                {get_perc("BH_down_line"):.0f}: BH_Downline {{ ball = {bh_downline}}} -> {opp_player}_de_stroke
                {get_perc("BH_down_mid"):.0f}: BH_DownMid {{ ball = {bh_downmid} }} -> {opp_player}_mid_stroke
                {get_perc("FH_inside_out"):.0f}: FH_InsideOut {{ ball = {fh_insideout}}} -> {opp_player}_ad_stroke
                {get_perc("FH_inside_in"):.0f}: FH_InsideIn {{ ball = {fh_insidein}}} -> {opp_player}_de_stroke
                {get_perc("FH_down_mid"):.0f}: FH_DownMid {{ ball = {fh_downmid}}} -> {opp_player}_mid_stroke
                {get_perc("FH_Error"):.0f}: FH_Error {{ ball = 9}} -> {{{opp_player_short}score++; if ({opp_player_short}score == 7) {{won = {opp_player_long}}}
                                                                             else {{ turn = (turn+1)%4 }}
                                                                             }} -> NextPt
                {get_perc("BH_Error"):.0f}: BH_Error {{ ball = 9}} -> {{{opp_player_short}score++; if ({opp_player_short}score == 7) {{won = {opp_player_long}}}
                                                                             else {{ turn = (turn+1)%4 }}
                                                                                 }} -> NextPt
                }};
            ''')

In [65]:
def wrapper_rally_pcsp(p1,p2,df):
    print(p1 + ':\n')
    overall_df = filter_df_rally(p1,p2,df)
    gen_rally_pcsp("Ply1","de",overall_df)
    gen_rally_pcsp("Ply1","mid",overall_df)
    gen_rally_pcsp("Ply1","ad",overall_df)        
    
    print(p2 + ':\n')
    overall_df = filter_df_rally(p2,p1,df)
    gen_rally_pcsp("Ply2","de",overall_df)
    gen_rally_pcsp("Ply2","mid",overall_df)
    gen_rally_pcsp("Ply2","ad",overall_df)        

wrapper_rally_pcsp("Roger Federer","Rafael Nadal",rally_133)

Roger Federer:

Ply1_de_stroke = pcase{
            50: FH_Crosscourt { ball = 6} -> Ply2_de_stroke
            16: FH_Downline { ball = 4} -> Ply2_ad_stroke
            15: FH_DownMid { ball = 5} -> Ply2_mid_stroke
            1: BH_InsideIn {ball = 4} -> Ply2_ad_stroke
            1: BH_InsideOut { ball = 6} -> Ply2_de_stroke
            1: BH_DownMid { ball = 5} -> Ply2_mid_stroke
            16: FH_Error { ball = 9} -> {nscore++; if (nscore == 7) {won = player2}
                else { turn = (turn+1)%4 }
                } -> NextPt
            0: BH_Error { ball = 9} -> {nscore++; if (nscore == 7) {won = player2}
                else { turn = (turn+1)%4 }
                } -> NextPt
            };
            
Ply1_mid_stroke = pcase{
            22: FH_InsideOut{ball = 4} -> Ply2_ad_stroke
            31: FH_Crosscourt { ball = 6} -> Ply2_de_stroke
            12: FH_DownMid { ball = 5 } -> Ply2_mid_stroke
            8: BH_InsideOut { ball = 6} -> Ply2_de_stroke
            8: BH

In [66]:
data_133[data_133["Player 1 name"].isin(list(rally_cluster_agglom[rally_cluster_agglom["Return Cluster"]==1]["Player"]))]["Player 1 handness"].value_counts()

LH    199276
Name: Player 1 handness, dtype: int64

In [67]:
# Rally Cluster # 1 (nadal's cluster) is ALL left-handed players
for clust in [0,1,2,3]:
    print(clust)
    print(data_133[data_133["Player 1 name"].isin(list(rally_cluster_agglom[rally_cluster_agglom["Return Cluster"]==clust]["Player"]))]["Player 1 handness"].value_counts())

0
RH    933343
Name: Player 1 handness, dtype: int64
1
LH    199276
Name: Player 1 handness, dtype: int64
2
RH    202123
Name: Player 1 handness, dtype: int64
3
RH    106150
Name: Player 1 handness, dtype: int64


In [68]:
# rally_cluster_agglom[rally_cluster_agglom["Player"]=="Rafael Nadal"]
list(rally_cluster_agglom[rally_cluster_agglom["Return Cluster"]==1]["Player"])

['Rafael Nadal',
 'Guido Pella',
 'Thomas Muster',
 'Albert Ramos',
 'Marcelo Rios',
 'Mischa Zverev',
 'John Mcenroe',
 'Fernando Verdasco',
 'Cameron Norrie',
 'Adrian Mannarino',
 'Martin Klizan',
 'Yoshihito Nishioka',
 'Jimmy Connors',
 'Federico Delbonis',
 'Denis Shapovalov',
 'Gilles Muller',
 'Feliciano Lopez',
 'Goran Ivanisevic']

Find a cluster that is Not Nadal nor Federer, and use that cluster inplace of Nadal's (Ply2's) cluster, to act as a comparison point / control variable?

In [91]:
rally_cluster_agglom[rally_cluster_agglom["Player"].isin(['Rafael Nadal','Roger Federer'])]

,Return Cluster,Player
6,1,Rafael Nadal
42,0,Roger Federer


In [108]:
rally_cluster_agglom.tail(20)

,Return Cluster,Player
101,0,Kyle Edmund
102,0,Jeremy Chardy
103,0,Nikoloz Basilashvili
104,0,Richard Krajicek
105,3,Pete Sampras
106,0,Andre Agassi
107,1,Gilles Muller
108,0,Jan Lennard Struff
109,1,Feliciano Lopez
110,0,Sam Querrey


We use Daniil Medvedev for cluster 2.
Because there can be too many combinations, given 3 game components, we will choose a 'Non-Ply2 cluster' by random for each Game Component. Given enough time, we could test more combinations.

Sampras is cluster 3

In [109]:
# wrapper_rally_pcsp("Roger Federer","Daniil Medvedev",rally_133)
# wrapper_rally_pcsp("Daniil Medvedev","Rafael Nadal",rally_133)
wrapper_rally_pcsp("Daniil Medvedev","Pete Sampras",rally_133)

Daniil Medvedev:

Ply1_de_stroke = pcase{
            33: FH_Crosscourt { ball = 6} -> Ply2_de_stroke
            25: FH_Downline { ball = 4} -> Ply2_ad_stroke
            21: FH_DownMid { ball = 5} -> Ply2_mid_stroke
            0: BH_InsideIn {ball = 4} -> Ply2_ad_stroke
            0: BH_InsideOut { ball = 6} -> Ply2_de_stroke
            0: BH_DownMid { ball = 5} -> Ply2_mid_stroke
            20: FH_Error { ball = 9} -> {nscore++; if (nscore == 7) {won = player2}
                else { turn = (turn+1)%4 }
                } -> NextPt
            0: BH_Error { ball = 9} -> {nscore++; if (nscore == 7) {won = player2}
                else { turn = (turn+1)%4 }
                } -> NextPt
            };
            
Ply1_mid_stroke = pcase{
            18: FH_InsideOut{ball = 4} -> Ply2_ad_stroke
            20: FH_Crosscourt { ball = 6} -> Ply2_de_stroke
            11: FH_DownMid { ball = 5 } -> Ply2_mid_stroke
            8: BH_InsideOut { ball = 6} -> Ply2_de_stroke
            15:

### data checks

In [36]:
p1="Roger Federer"
p2 = "Rafael Nadal"

cluster_no = rally_cluster_agglom[rally_cluster_agglom["Player"]==p1]["Return Cluster"].values[0]
opp_cluster_no = rally_cluster_agglom[rally_cluster_agglom["Player"]==p2]["Return Cluster"].values[0]

player_cluster_df = rally_133[rally_133["Player 1 name"].isin(list(rally_cluster_agglom[rally_cluster_agglom["Return Cluster"]==cluster_no]["Player"]))]
player_v_opp_player_df = player_cluster_df[player_cluster_df["Player 2 name"].isin(list(rally_cluster_agglom[rally_cluster_agglom["Return Cluster"]==opp_cluster_no]["Player"]))]
# player_v_opp_player_df


# print(from_court)
# temp_dict = {}
from_court=list(player_v_opp_player_df['From_Court'].unique())
for fc in from_court:
    print(player_v_opp_player_df[player_v_opp_player_df['From_Court']== fc].groupby(['From_Court','rally_outcome']).size().transform(lambda x: x/x.sum()))
    # temp_dict["{}".format(fc)] = temp_df


From_Court    rally_outcome 
middle court  -                 0.000283
              BH_Error          0.034970
              BH_None           0.000324
              BH_cross_court    0.073582
              BH_down_mid       0.082082
              BH_inside_out     0.075201
              FH_Error          0.086818
              FH_None           0.000364
              FH_cross_court    0.311410
              FH_down_mid       0.116769
              FH_inside_out     0.218197
dtype: float64
From_Court  rally_outcome 
ad court    -                 0.000358
            BH_Error          0.131953
            BH_None           0.000912
            BH_cross_court    0.275759
            BH_down_line      0.199394
            BH_down_mid       0.233913
            FH_Error          0.020581
            FH_None           0.000163
            FH_down_mid       0.025596
            FH_inside_in      0.054318
            FH_inside_out     0.057054
dtype: float64
From_Court   rally_outcome 
deuce 

In [37]:
list(player_v_opp_player_df['From_Court'].unique())

['middle court', 'ad court', 'deuce court', 'unknown']

## EDA?

In [38]:
print(djokovic_nadal_total[(djokovic_nadal_total["Player 1 name"] == "Novak Djokovic") & (djokovic_nadal_total["Shot Type"]==2) & (djokovic_nadal_total["From which court"]==1)]["Shot_Outcome_group"].value_counts(normalize=True))
print(djokovic_nadal_total[(djokovic_nadal_total["Player 1 name"] == "Novak Djokovic") & (djokovic_nadal_total["Shot Type"]==2) & (djokovic_nadal_total["From which court"]==1)]["Shot_Outcome_group"].value_counts(normalize=True).plot(kind="bar"))
# serve0_overall[(serve0_overall["Player 1 name"].isin(serve_0_list)) & (serve0_overall["Shot Type"]==1)]["Shot outcome"].value_counts(normalize=True).plot(kind="bar")
# match1[(match1["Player 1 name"]== "Novak Djokovic") & (match1["Shot Type"]==1)]["Shot outcome"].value_counts().plot(kind='bar')

NameError: name 'djokovic_nadal_total' is not defined

In [ ]:
#Outcome probabilities
# 35% fail, 65% success
fail = 0.35
success = 0.65

#Serve directions
serve_T = 23+8
serve_wide = 26+32
serve_body = 9+2

# Serve T: 9%
serveT_in = serve_T*success*10
print(serveT_in)
serveT_out = serve_T*fail*10
print(serveT_out)

# Serve Wide: 39%
serveWide_in = serve_wide*success*10
print(serveWide_in)
serveWide_out = serve_wide*fail*10
print(serveWide_out)

# Serve Body: 52%
serveBody_in = serve_wide*success*10
print(serveBody_in)
serveBody_out = serve_body*fail*10
print(serveBody_out)

Overall, Djokovic 2nd-serve results (from Ad Court) to Nadal, from all matches:

In [ ]:
print(djokovic_nadal_total[(djokovic_nadal_total["Player 1 name"] == "Novak Djokovic") & (djokovic_nadal_total["Shot Type"]==2) & (djokovic_nadal_total["From which court"]==3)]["Shot_Outcome_group"].value_counts(normalize=True))
print(djokovic_nadal_total[(djokovic_nadal_total["Player 1 name"] == "Novak Djokovic") & (djokovic_nadal_total["Shot Type"]==2) & (djokovic_nadal_total["From which court"]==3)]["Shot_Outcome_group"].value_counts(normalize=True).plot(kind="bar"))
# serve0_overall[(serve0_overall["Player 1 name"].isin(serve_0_list)) & (serve0_overall["Shot Type"]==1)]["Shot outcome"].value_counts(normalize=True).plot(kind="bar")
# match1[(match1["Player 1 name"]== "Novak Djokovic") & (match1["Shot Type"]==1)]["Shot outcome"].value_counts().plot(kind='bar')

In [ ]:
test_df = djokovic_nadal_total[(djokovic_nadal_total["Player 1 name"] == "Novak Djokovic") & (djokovic_nadal_total["Shot Type"]==2) & (djokovic_nadal_total["From which court"]==3)]["Shot_Outcome_group"].value_counts(normalize=True)
test_df

In [ ]:
djoko_ad_2nd_serve = {}
for idx,value in enumerate(test_df):
    print(idx)
    print(value)
#     print(value)
#     print('idx',(djokovic_nadal_total[(djokovic_nadal_total["Player 1 name"] == "Novak Djokovic") & (djokovic_nadal_total["Shot Type"]==2) & (djokovic_nadal_total["From which court"]==3)]["Shot_Outcome_"].value_counts(normalize=True)))
#     print(f'Percentage : {value:.3f}')
#     print('Percentage :', serve0_overall[(serve0_overall["Player 1 name"].isin(serve_0_list)) & (serve0_overall["Shot Type"]==1)]["Shot_Outcome_"].value_counts(normalize=True)[idx])
#     percentage = serve0_overall[(serve0_overall["Player 1 name"].isin(serve_0_list)) & (serve0_overall["Shot Type"]==1)]["Shot_Outcome_"].value_counts(normalize=True)[idx]
#     serve0_dict[name] = percentage

In [ ]:
#Outcome probabilities
fail = 0.08
success = 0.92

#Serve directions
serve_T = 40+3
serve_wide = 16+2
serve_body = 39+0

# Serve T: 9%
serveT_in = serve_T*success*10
print(serveT_in)
serveT_out = serve_T*fail*10
print(serveT_out)

# Serve Wide: 39%
serveWide_in = serve_wide*success*10
print(serveWide_in)
serveWide_out = serve_wide*fail*10
print(serveWide_out)

# Serve Body: 52%
serveBody_in = serve_wide*success*10
print(serveBody_in)
serveBody_out = serve_body*fail*10
print(serveBody_out)